In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import os

In [2]:
df = pd.read_feather('cleaned_seqs.df').set_index('index').query("ds_type == 'test'")
func = lambda s: s[s.find("images/")+7:]
df["local_path"] = np.array([[f"../seq_data/{s}/{func(i)}"for i in c] for c,s in zip(df.Paths,df.ds_type)]).tolist()

In [9]:
!mkdir tmp/test_data/front/
!mkdir tmp/test_data/side/
!mkdir tmp/test_data/new_front/
!mkdir tmp/test_data/new_side/


In [6]:
!rm tmp/test_data/front/*

In [5]:
hhh = 0
for x,y in tqdm.tqdm(df.groupby(["seq_key","og_cluster"])):
    row = y.iloc[0]

    if hhh< 720:
        hhh += 1
        continue

    hhh +=1
    fig, ax = plt.subplots(2,2,figsize=(12,12))
    keys = row['local_path']
    #np.random.shuffle(keys)
    for i,a in enumerate(ax.flatten()):
        try:
            a.imshow(plt.imread(keys[i]))
        except: pass
        a.axis('off')
    plt.suptitle(f"{row['view_direction']}")
    if row['view_direction'] == 'Forward':
        path = "tmp/test_data/front/" +  x[0]+"Ä" +str(x[1]) + ".jpg"
    else:
        path = "tmp/test_data/side/" +  x[0]+"Ä" +str(x[1]) + ".jpg"

    plt.savefig(path)
    plt.close()


100%|██████████| 2295/2295 [05:33<00:00,  6.88it/s] 


In [3]:
len(os.listdir('tmp/test_data/front/'))

1941

In [4]:
sorted(os.listdir('tmp/test_data/front/')).index("le8ayfdqjthe0si96h1crpÄ711.jpg")

1282

In [3]:
# inspect = [i[:-4] for i in os.listdir("tmp/test_data/new_front/")]
# import os
# os.makedirs("tmp/test_data/inspect",exist_ok=True)
# for c in tqdm.tqdm(inspect):
#     eins,zwei = c.split('Ä')
#     vv = df[(df["og_cluster"] == int(zwei)) & (df["seq_key"] == eins)]
#     os.makedirs(f"tmp/test_data/inspect/{c}",exist_ok=True)
#     for d in range(len(vv)):
#         row = vv.iloc[d]
#         fig, ax = plt.subplots(2,2,figsize=(12,12))
#         keys = row['local_path']
#         #np.random.shuffle(keys)
#         for i,a in enumerate(ax.flatten()):
#             try:
#                 a.imshow(plt.imread(keys[i]))
#             except: pass
#             a.axis('off')
#         plt.suptitle(f"{row['view_direction']}")
#         plt.savefig(f"tmp/test_data/inspect/{c}/{vv.index[d]}.jpg")
#         plt.close()

    

100%|██████████| 26/26 [00:44<00:00,  1.71s/it]


In [ ]:
new

In [15]:
dels_f = [x for x,y in tqdm.tqdm(df.groupby(["seq_key","og_cluster"])) if y.iloc[0].view_direction =="Forward" and x[0]+"Ä" + str(x[1])+".jpg" not in os.listdir('tmp/test_data/front')]
dels_s = [x for x,y in tqdm.tqdm(df.groupby(["seq_key","og_cluster"])) if y.iloc[0].view_direction =="Sideways" and x[0]+"Ä" + str(x[1])+".jpg" not in os.listdir('tmp/test_data/side')]

100%|██████████| 2295/2295 [00:00<00:00, 3751.75it/s]


In [17]:
del_s = ([x for x in dels_s if x[0]+"Ä" + str(x[1])+".jpg" not in os.listdir('tmp/test_data/new_front')])

In [24]:
dels_s.extend(dels_f)

In [29]:
index = np.concatenate([df.loc[(df['seq_key']==seq) & (df['cluster_id']==cl)].index for seq,cl in dels_s])

164

In [30]:
df.drop(index = index, inplace= True)

In [38]:
front = [i[:-4].split('Ä') for i in os.listdir('tmp/test_data/new_front/')]
for seq,cl in front:
    df.loc[(df['seq_key']==seq) & (df['cluster_id']==cl),"view_direction"] == "Forward"

In [39]:
df.value_counts('view_direction')

view_direction
Forward     2228
Sideways    1920
Backward       1
dtype: int64

In [40]:
df.reset_index().to_feather("test.df")